# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

## Data Prep

In [1]:
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_cases').getOrCreate()
from pyspark.ml.regression import LinearRegression

In [2]:
df = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

In [7]:
for i in df.head(1)[0]:
    print(i)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [5]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [6]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



## FE

In [42]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

First, index the Cruise_line variable using stringIndexer.https://spark.apache.org/docs/latest/ml-features#stringindexer

In [39]:
# cl = [i[0] for i in list(df.select("Cruise_line").distinct().collect())]
# index = range(len(cl))
# result_list = list(zip(cl, index))

In [50]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_Index")
indexed = indexer.fit(df).transform(df)
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_Index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|              1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|              1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|       

Transform all features to a vector.

In [52]:
assembler = VectorAssembler(
  inputCols=['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density',
             'Cruise_line_Index'],
    outputCol="features")
output = assembler.transform(indexed)

In [53]:
output.select("features", "crew").show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [55]:
final_data = output.select("features", "crew")

## Modeling

In [56]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [57]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              122|
|   mean|7.707295081967224|
| stddev|3.648039672124896|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [59]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                36|
|   mean|  8.08861111111111|
| stddev|2.9892685706208444|
|    min|               1.6|
|    max|             13.13|
+-------+------------------+



In [63]:
lr = LinearRegression(labelCol='crew')

In [64]:
lrModel = lr.fit(train_data)

In [65]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.014188437458238804,0.015756108152268092,-0.18191915552853782,0.4208859544853525,0.8706570357291208,-0.006432522747790024,0.03671270916752314] Intercept: -0.7211481344990991


In [66]:
test_results = lrModel.evaluate(test_data)

In [67]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -1.4202043089192333|
|  0.1278057963095236|
| -0.7622656111056125|
| 0.46604388804234453|
|  -0.636309818340262|
| -1.3604252866437925|
|  0.8328599630611535|
| -0.4715710450588375|
| -0.4009331012626607|
|  0.7597991476333963|
| -1.2778793071850583|
|  0.6615173369797755|
|0.052750342364324965|
| -1.3320484117273157|
|0.002605764547980449|
| -0.6438919023906786|
| -1.5251309334133554|
| 0.14029620206008042|
| -1.3936715368108388|
|  0.2243756812941129|
+--------------------+
only showing top 20 rows



In [69]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 0.7802384509519049
MSE: 0.6087720403438281
R2: 0.9299256170057141
